In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving text_test1.txt to text_test1.txt
Saving text_train1.txt to text_train1.txt


In [ ]:
data_file = open('dataFinal.json').read()
intents = json.loads(data_file)

In [ ]:
print(intents)

In [ ]:
pickle.dump(intents,open('intentsRnn.pkl','wb'))

In [ ]:
from google.colab import files
files.download('intentsRnn.pkl')

In [ ]:
for intent in intents['intents']:
    print(intent['patterns'])
    break

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
print(documents[50])

(['Quel', 'est', 'ton', 'numero'], '3')


In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))



96 documents
54 classes ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '5', '6', '7', '8', '9', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
164 unique lemmatized words [',', '.', 'a', 'adresse', 'adverse', 'age', 'aide', 'all', 'allez-vous', 'as-tu', 'au', 'avec', 'behavior', 'bientôt', 'blood', 'bonjour', 'bonne', 'bonsoir', 'by', "c'est", 'ca', 'causing', 'ce', 'centre', 'check', 'chef', 'comment', 'comprenez-vous', 'connaitre', 'content', "d'interets", "d'où", 'data', 'de', 'derange', 'detail', 'discussion', 'dont', 'drug', 'du', 'enchanté', 'enchantée', 'entry', 'es-tu', 'est', 'est-ce', 'faire', 'find', 'for', 'fournissez', 'français', 'freres', 'give', 'génial', 'have', 'hey', 'history', 'hola', 'ho

In [ ]:
from google.colab import files
files.download('wordsRnn.pkl')
files.download('classesRnn.pkl')
files.download('documentsRnn.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 from google.colab import files
uploaded = files.upload()

Saving classes.pkl to classes.pkl
Saving documents.pkl to documents.pkl
Saving words.pkl to words.pkl


In [ ]:
infile = open('words.pkl','rb')
words = pickle.load(infile)
infile.close()

In [ ]:
infile2 = open('classes.pkl','rb')
classes = pickle.load(infile2)
infile2.close()

In [ ]:
infile3 = open('documents.pkl','rb')
documents = pickle.load(infile3)
infile3.close()

In [ ]:
print(documents[1000])

In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print(len(train_y[0]))
print(len(train_x[0]))

Training data created
54
164


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
pickle.dump(model,open('modelRnn.pkl','wb'))

print("model created")

In [ ]:
with open('modelRnn.pkl', 'wb') as file:
        pickle.dump(model, file)
        

In [ ]:
from threading import Thread 
def hanlder(filename1,filename2,filename3):
    with open('modelRnn.pkl', 'wb') as file:
        pickle.dump(filename1, file)
        pickle.dump(filename2, file)
        pickle.dump(filename3, file)


t = Thread(target=hanlder, args=(model))
t.start()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: hanlder() argument after * must be an iterable, not Sequential



In [ ]:
from google.colab import files
files.download('chatbot_model.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               21120     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 54)                3510      
Total params: 32,886
Trainable params: 32,886
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("Evaluate on test data")
results = model.evaluate(np.array(train_x), np.array(train_x))[1]
print("test loss, test acc:", results)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving chatbot_model.h5 to chatbot_model.h5


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
msg = str(input('Write a question :'))
print(chatbot_response(msg))

Write a question :merci
À tout moment!
